In [ ]:
!git clone https://github.com/andreazenotto/tempProjectRepo.git

In [ ]:
%%capture
!apt update && apt install -y openslide-tools
!pip install openslide-python

In [ ]:
import gdown
import os
from tqdm import tqdm
import pandas as pd
import sys

sys.path.append('tempProjectRepo/mesothelioma_project/src')
from wsi_utils import load_wsi, extract_patches, count_patches

## Count patches

In [ ]:
data_csv = "tempProjectRepo/mesothelioma_project/data/raw-data.csv"
output_dir = "tempProjectRepo/mesothelioma_project/data/patches"

diagnosis_map = {"E": "epithelioid", "S": "sarcomatoid", "B": "biphasic"}

df = pd.read_csv(data_csv, delimiter=r"\s+")
start_idx = 0
end_idx = len(df)

count_dict = {}

for _, row in tqdm(df.iloc[start_idx:end_idx].iterrows(), total=len(df)):
    filename = row['filename']
    gdrive_id = row['id']
    diagnosis_code = row['diagnosis']
    diagnosis_name = diagnosis_map[diagnosis_code]

    name = filename.split(".")[0] + " - " + diagnosis_name
    count_dict[name] = 0

    wsi_url = f"https://drive.google.com/uc?id={gdrive_id}"
    gdown.download(wsi_url, quiet=False)

    slide_id = os.path.splitext(filename)[0]
    slide_output_dir = os.path.join(output_dir, diagnosis_name, slide_id)

    # Load and process the WSI
    slide = load_wsi(filename)
    count_dict[filename] = count_patches(slide, slide_output_dir, level=1)

# Sort the dictionary by values in descending order
sorted_count_dict = dict(sorted(count_dict.items(), key=lambda item: item[1], reverse=True))

## Segmentation and Patching

In [ ]:
data_csv = "tempProjectRepo/mesothelioma_project/data/raw-data.csv"
output_dir = "tempProjectRepo/mesothelioma_project/data/patches"

diagnosis_map = {"E": "epithelioid", "S": "sarcomatoid", "B": "biphasic"}

df = pd.read_csv(data_csv, delimiter=r"\s+")
start_idx = 0
end_idx = len(df)

for _, row in tqdm(df.iloc[start_idx:end_idx].iterrows(), total=len(df)):
    filename = row['filename']
    gdrive_id = row['id']
    diagnosis_code = row['diagnosis']
    diagnosis_name = diagnosis_map[diagnosis_code]

    wsi_url = f"https://drive.google.com/uc?id={gdrive_id}"
    gdown.download(wsi_url, quiet=False)

    slide_id = os.path.splitext(filename)[0]
    slide_output_dir = os.path.join(output_dir, diagnosis_name, slide_id)

    # Load and process the WSI
    slide = load_wsi(filename)
    extract_patches(slide, slide_output_dir, level=1, threshold=30)

In [ ]:
from google.colab import files
import shutil

# Percorso della cartella da scaricare
folder_path = "tempProjectRepo/mesothelioma_project/data/patches"

# Nome del file zip da creare
zip_filename = "patches.zip"

# Comprimi la cartella
shutil.make_archive(zip_filename.replace(".zip", ""), 'zip', folder_path)

# Scarica il file zip
files.download(zip_filename)